Code-snippet to access the OPEN AI API

In [32]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY') # Remember to save the OpenAI API key as an environment variable under the name of "OPENAI_API_KEY"


Setting up the text-analyzer, using the template from Deeplearning:

In [33]:
def get_TextEvaluation(text, prompt, model = "gpt-4", temperature = 0, max_tokens = 400):
    messages = [{"role": "system"   , "content": prompt}, 
                {"role": "user"     , "content": text}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature, # this is the degree of randomness of the model's output
        max_tokens = max_tokens,
    )
    return response.choices[0].message["content"]

Setting up the rating-analyzer:

In [34]:
def get_RiskEvaluation(text, prompt, model = "gpt-4", temperature = 0, max_tokens = 300):
    messages = [{"role": "system"   , "content": prompt}, 
                {"role": "user"     , "content": text}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature, # this is the degree of randomness of the model's output
        max_tokens = max_tokens,
    )
    return response.choices[0].message["content"]

Setting the criterias for evaluation:

In [35]:
criteria = """
Grønnvaskingsplakaten punkt 1: Vær ærlig og etterrettelig. Ikke bruk misvisende ord eller bilder, og ikke tilpass resultater fra tester og undersøkelser til egen fordel.
Grønnvaskingsplakaten punkt 4: Unngå å skjule eller nedtone egne utslipp og negativ innvirkning på klima, natur og mennesker.
Grønnvaskingsplakaten punkt 8: Vær forsiktig med uttrykk som “bedre for klimaet”, “bedre for naturen”, “bedre for miljøet”, etc. hvis du ikke kan vise til konkrete tall og resultater.
Grønnvaskingsplakaten punkt 10: Vær forsiktig med å bruke donasjoner og sponsorater som bevis på at du jobber med bærekraft.
"""

Setting the baseline text and rating evaluation prompts:

In [36]:
text_eval_prompt = f"""
Vurder hvor godt teksten oppfyller kravene i punkt 1, 4, 8, og 10 i Grønnvaskingsplakaten. \
Gi en overordnet tilbakemelding for grønnvasking basert på teksten, samt en individuell tilbakemelding for forbedringspunkter for hvert av de fire punktene. \
Hvis du er usikker, skal du respondere med "Hmm.. Dette vet jeg ikke" og ikke improvisere et svar.

Hvis teksten tilsynelatende ikke har noe med bærekraftsarbeid i organisasjoner å gjøre, skriv "Hmm.. Dette er ikke en tekst jeg gjenkjenner. Prøv igjen."

Legg spesielt vekt på om teksten gir konkrete tall eller detaljer til sine utsagn. 

Punktene i grønnvaskingsmodellen er: ```{criteria}```

Bruk maks 350 ord
"""

In [37]:
risk_prompt = f"""
Vurder hvor godt teksten oppfyller kravene i punkt 1, 4, 8, og 10 i Grønnvaskingsplakaten. \
Gi en overordnet risikovurdering for grønnvasking basert på teksten, samt en individuell risikovurdering for hvert av de fire punktene. \
Hvis du er usikker, skal du respondere med "Hmm.. Dette vet jeg ikke" og ikke improvisere et svar.

Hvis teksten tilsynelatende ikke har noe med bærekraftsarbeid i organisasjoner å gjøre, skriv "Hmm.. Dette er ikke en tekst jeg gjenkjenner. Prøv igjen."

Bruk strenge kriterier for risikovurderingene; det krever mye for å oppnå "Lav" og tilsvarende ekstremt god dokumentasjon for å oppnå "Ingen". \
Legg spesielt vekt på om teksten gir konkrete tall eller detaljer til sine utsagn. Returner svaret i følgende format:

Overordnet risiko for grønnvasking: (Ikke relevant / Ingen / Lav / Moderat / Høy)
Risiko for brudd på punkt 1: (Ikke relevant / Ingen /  Lav / Moderat / Høy)
Risiko for brudd på punkt 4: (Ikke relevant / Ingen /  Lav / Moderat / Høy)
Risiko for brudd på punkt 8: (Ikke relevant / Ingen /  Lav / Moderat / Høy)
Risiko for brudd på punkt 10: (Ikke relevant / Ingen /  Lav / Moderat / Høy) 

Punktene i grønnvaskingsmodellen er: ```{criteria}```

Bruk maks 100 ord.
"""

Inserting example-texts from user-message:

In [38]:
high = 'Vi introduserer vår revolusjonerende "EcoPower Batteri", det \
mest miljøvennlige batteriet på markedet! Vi har harneskert uendelig energi \
fra Moder Jord selv for å skape dette magiske produktet. Hver EcoPower-batteri \
er infusert med grønn kraft som er hentet direkte fra naturen. Når du velger \
EcoPower, velger du å redde planeten vår. \
Vårt EcoPower Batteri har mottatt mange priser og anerkjennelse \
for sitt banebrytende arbeid innen miljøvern. \
Ingen andre batterier kan sammenlignes med den grønne \
kraften til EcoPower. Ved å bytte til vårt batteri, \
kan du være trygg på at du gjør det absolutt beste \
for miljøet. La oss sammen kjempe for en grønnere \
fremtid med EcoPower Batteri!'

In [39]:
low = 'Vårt nye produkt, "PureGreen Vaskepulver", er resultatet av mange års forskning og utvikling innen miljøvennlig rengjøringsteknologi. \
Vi har klart å skape et vaskemiddel som ikke bare er effektivt på tøffe flekker, men som også er snill mot miljøet. PureGreen inneholder \
biologisk nedbrytbare ingredienser som er hentet på en bærekraftig måte. Hver ingrediens i vår formel er klart listet opp på pakken, sammen \
med en forklaring på hvordan den bidrar til miljøvennligheten av produktet.PureGreen Vaskepulver er produsert i vår CO2-nøytrale fabrikk, \
noe vi er svært stolte av. Alle våre miljøvennlige tiltak, fra produksjonsprosessen til forpakningen, er dokumentert og verifisert av \
anerkjente miljøsertifiseringsorganisasjoner. Sertifiseringene inkluderer Green Seal og EU Ecolabel, som er anerkjent for strenge miljøstandarder. \
Vi deler transparent våre fremgangsmåter og vi unngår å overdrive de positive effektene av vårt produkt. Vi er dedikerte til å være åpne og \
ærlige med våre kunder, slik at du kan føle deg trygg på at når du velger PureGreen, tar du et virkelig miljøvennlig valg.'

In [40]:
irrelevant = 'Fra øyeblikket jeg steg inn i La Bella Vita, ble jeg raskt tatt med til de vakre landskapene i Italia, både gjennom den varme \
og innbydende atmosfæren og de deilige duftene som fylte luften. Plassert i hjertet av Bergen, er denne restauranten et autentisk uttrykk for \
italiensk mat og kultur. Maten her er ikke mindre enn utmerket. Fra de rike og robuste smakene av deres signatur osso buco, til den friske og \
levende Caprese-salaten, klarte hvert rett å imponere. Den hjemmelagde pastaen er et must-try, perfekt tilberedt al dente hver gang, og badet i \
en rekke sauser som får munnen til å vann. Vinkartet til La Bella Vita er omfattende, med flotte alternativer fra alle regioner i Italia. \
De har også en imponerende samling av grappa og amari, for de som ønsker å avslutte måltidet på ekte italiensk vis.'

In [41]:
def trigger(text, text_prompt, rist_prompt):
    output2 = get_TextEvaluation(text, text_prompt)

    output1 = get_RiskEvaluation(text, risk_prompt)
    
    if output1 == "Hmm.. Dette er ikke en tekst jeg gjenkjenner. Prøv igjen.":
        return output1
    else:
        return output1 + "\n\n" + output2

Printing Examples:

In [42]:
print(trigger(low, text_eval_prompt, risk_prompt))


Overordnet risiko for grønnvasking: Lav
Risiko for brudd på punkt 1: Lav
Risiko for brudd på punkt 4: Lav
Risiko for brudd på punkt 8: Lav
Risiko for brudd på punkt 10: Ikke relevant

Overordnet sett oppfyller teksten kravene i Grønnvaskingsplakaten godt. Den gir konkrete detaljer og tall, og unngår å bruke misvisende ord eller bilder.

Punkt 1: Teksten er ærlig og etterrettelig. Den gir konkrete detaljer om produktet, inkludert ingrediensene og hvordan de bidrar til miljøvennligheten. Den nevner også at produktet er verifisert av anerkjente miljøsertifiseringsorganisasjoner. For å forbedre, kunne teksten ha inkludert mer detaljert informasjon om testresultater og undersøkelser.

Punkt 4: Teksten unngår å skjule eller nedtone egne utslipp og negativ innvirkning på klima, natur og mennesker. Den nevner at produktet er produsert i en CO2-nøytral fabrikk. For å forbedre, kunne teksten ha inkludert mer informasjon om eventuelle negative effekter av produksjonen.

Punkt 8: Teksten er forsik